In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt

In [2]:
def calc_auv2_accel(T, alpha, theta, mass=100):
    """given a np array of forces in newtons, an alpha offset of the motor in radians, and a theta offset of the vehicle in radians, and the mass of the vehicle  in kgs, reutrns the a 2d array of the vehicle's accleration in m/s^2"""
    if mass <= 0:
        raise ZeroDivisionError("can't divide by 0/negative mass")
    Thorz = T * math.cos(alpha)
    Tvert = T * math.sin(alpha)
    tothorizontal = Thorz[0] + Thorz[1] - Thorz[2] - Thorz[3]
    totvertical = Tvert[0] - Tvert[1] - Tvert[2] + Tvert[3]
    accel = np.array([tothorizontal, totvertical])
    rotate = np.array(
        [[math.cos(theta), -math.sin(theta)], [math.sin(theta), math.cos(theta)]]
    )
    return np.dot(rotate, accel) / mass


def calc_auv2_ang_accel(T, alpha, L, l, interia=100):
    Thorz = T * np.sin(alpha)
    Tvert = T * np.cos(alpha)
    torques = l * Thorz + L * Tvert
    return torques[0] - torques[1] + torques[2] - torques[3]


def sim_auv_motion(
    T, alpha, L, l, lmass, inertia, dt=0.1, t_final=10, x0=0, y0=0, theta=0
):
    count = 0
    curang = theta
    timestamps = np.array([0])
    x = np.array([x0])
    y = np.array([y0])
    theta = np.array([theta])
    v = np.array([0, 0])
    omega = np.array([0])
    a = np.array([0, 0])
    while count <= t_final:
        count += dt
        timestamps = np.append(timestamps, count)
        accel = calc_auv2_accel(T, alpha, curang)
        a = np.append(a, accel)
        angaccel = calc_auv2_ang_accel(T, alpha, L, l, inertia)
        newv = v[-1] + accel * dt
        newomega = omega[-1] + angaccel * dt
        v = np.append(v, newv)
        omega = np.append(omega, newomega)
        newtheta = theta[-1] * dt + 0.5 * angaccel * dt * dt
        theta = np.append(theta, newtheta)
        curang = theta
        newcoords = np.array([x[-1], y[-1]]) * dt + 0.5 * accel * dt * dt
        x = np.append(x, newcoords[0])
        y = np.append(y, newcoords[1])
    return timestamps, x, y, theta, v, omega, a


In [3]:
timestamps, x, y, theta, v, omega, a = sim_auv_motion(np.array([0,0,10,10]),  0, 3, 3, 3, 3)


TypeError: only length-1 arrays can be converted to Python scalars